# Chapter 7: Hypothesis and Inference

## Statistical Hypothesis Testing

Hypothesis are assertions like "This coin is fair", "Data Scientists prefer python to R", "Popups dissuade visitors to our site", etc, which can be translated into statistics about data. Under various assumptions, those statistics can be thought of as observations of random variables from known distributions, which allows us to make statements about how likely those assumtpions to hold. 

In the classic setup, we have a null hypothesis ${H_0}$, that represents some default position and some alternative hypothesis ${H_1}$ that we'd like to compare it with. We use statistcs to decide whether we can reject ${H_0}$ as false or not. 

### Example- Flipping a Coin. 

Imagine we have a coin, and we want to test whether it's fair. We make the assumption that the coin has some probability $p$ of landing on heads, so our null hypothesis is that the coin is fair- that $$p = 0.5$$. We'll test this agains the alternative hypothesis $$p \neq 0.5$$. 

In particular, our test will involve flipping the coin some number $n$ times and counting the number of heads. Each coin flip is a Bernoulli trial, which means that X is a Binomial(n,p) random variable which(as we saw in Chapeter 6), we can approximate using the normal distribution. 



In [35]:
import random
import math

import ipynb.fs.defs.Chapter6_Probability as prob

In [36]:
def normal_approximation_to_binomial(n,p):
    """finds mu and sigma corresponding to a Binomial (n,p)"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma



We can use the normal_cdf function to figure out the probability that a random variable (following a normal distriubution) lies inside or outside a particular interval. 


In [37]:
# the normal cdf is the probability the variable is below a threshold
normal_probability_below = prob.normal_cdf          # function synonym

# Considered above the threshold if it's not below: 
def normal_probability_above(lo, mu=0, sigma=1):
    return 1- prob.normal_cdf(lo, mu, sigma)

# It's between if it's less than hi but not less than lo. 
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return prob.normal_cdf(hi, mu, sigma) - prob.normal_cdf(lo, mu, sigma)

# It's outside if it's not between
def normal_probability_outside(lo, hi, mu, sigma):
    return 1 - normal_probability_between(lo, hi, mu, sigma)


Can also do the reverse- find the non-tail region or the symmetric interval around the mean that accounts for a certain level of likelihood. For example, if we want to find an interval centred at the mean and containing 60% probability, then we find the cutoffs where the upper and lower tail each contain 20% of the probability (thereby leaving 60%). 

In [38]:
def normal_upper_bound(probability, mu=0, sigma=1):
    """return the z for which P( Z <= z ) == probability"""
    return prob.inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
    """returns the z for which P( Z >= z ) == probability"""
    return prob.inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """return the symmetric (about the mean) bounds 
       that contain the specified probability """
    tail_probability = (1 - probability) / 2
    
    # Upper bound probability should have tail probability above it. 
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    
    # Lower bound probability should have tail probability above it. 
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound



Say we wanted to flip the coin 1000 times. If the coin was fair, X should be approximately normal with mean 500 and standard deviation 15.8. 

We need to make a decision about significance- how willing we are to make a type 1 error (false positive classification), in which we reject ${H_0}$ even though it is actually true. For reasons lost to time, the significance level is usually either 5% (1 in 20 type 1 error) or 1% (1 in 100 type 1 error). Choosing 5%: 



In [39]:
mu_0, sigma_0 = normal_approximation_to_binomial( 1000, 0.5)

# Determine the test that rejects H0 if X falls outside the bounds below: 

normal_two_sided_bounds( 0.95, mu_0, sigma_0)

(469.01026640487555, 530.9897335951244)

Assuming p really equals 0.5 (i.e. ${H_0}$ is true), there is just a 5% chance we observe an X that lies outside this interval, which is the exact significance we wanted.  Said differently, if ${H_0}$ is true then, approximately 19 times out of 20, this test will give the correct result. 

We're often interested in the power of a test, which is the probability of not making a type 2 error, in which we fail to reject ${H_0}$ even though it is really false. In order to measure this, we have to specify what exactly ${H_0}$ being false means- knowing that p is not 0.5 doesn't say anything abpout the distribution of X. In particular, lets check what happens if p is really 0.55, so the coin has a slight bias towards heads. 

The power of the test can be calculated as below. 

In [41]:
# 95% bounds based on assumption p is 0.5. 
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)
print(lo, hi)

# actual mu and sigma based on p=0.55
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

print(mu_1, sigma_1)

# A type 2 error means we fail to reject the null hypothesis (H0)
# which will happen when X is still in our original interval. 
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)

power = 1 - type_2_probability

print(power)

469.01026640487555 530.9897335951244
550.0 15.732132722552274
0.8865480012953671


Imagine instead that our null hypothesis ${H_0}$ was that the coin is not biased towards heads, or that $p \leq 0.5$. In that case, we want a one-sided test that rejects the null hypotheses when X is much larger than 500 but not when X is smaller than 500. So, a 5% significance test involves using normal_probability_below to find the cutoff below which 95% of the probabilities lie. 

In [43]:
hi= normal_upper_bound(0.95, mu_0, sigma_0)
print(hi)

type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
print(power)

526.0073585242053
0.9363794803307173


## p-values

p-values provide a different way to think about this. Instead of looking at broundaries based on a probability cutoff, we instead compute the probability (assuming ${H_0}$ is true) that we would see a value at least as extreme as the one we actually observed. 

For our two-sided test of whether the coin is fair, we could compute: 


In [45]:
def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu: 
        
        # if x is greater than the mean, the tail is what's greater than X...
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # x is less than the mean, the tail is what's below x. 
        return 2 * normal_probability_below(x, mu, sigma)
    
# e.g. p value for seeing 530 heads
two_sided_p_value(529.5, mu_0, sigma_0)
        

0.06207721579598857

### Note

The change from 530 to 529.5 is what's called a continuity correction. 
It reflects the fact that normal_probability_between(529.5, 350.5, mu_0, sigma_0) is a better estimate of the probability of seeing 530 heads than norma_probability_between(530,531,mu_0, sigma_0). 

Correspondingly, normal_probability_above(529.5, mu_0, sigma_0) is a better estimate of the probability of seeing at least 530 heads. 

In [47]:
# Simulation of this... 

extreme_value_count = 0
for _ in range(100000):
    num_heads = sum(1 if random.random() < 0.5 else 0    # count of heads
                   for _ in range(1000))                 # in 1000 flips
    if num_heads >= 530 or num_heads <= 470:              # count how often
        extreme_value_count += 1                         # we see extremes
        
print(extreme_value_count / 100000)

0.06104
